In [1]:
%reload_kedro

2023-09-13 22:06:33,857 - kedro.extras.extensions.ipython - INFO - No path argument was provided. Using: /Users/gpalazzo/Desktop/dev/crypto_msc_thesis
2023-09-13 22:06:34,496 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2023-09-13 22:06:34,789 - kedro.extras.extensions.ipython - INFO - ** Kedro project crypto_thesis
2023-09-13 22:06:34,790 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2023-09-13 22:06:34,800 - kedro.extras.extensions.ipython - INFO - Registered line magic `run_viz`


In [2]:
from crypto_thesis.utils import mt_split_train_test, scale_train_test
from crypto_thesis.data_domains.master_table.master_table import _build_window_numbers

In [3]:
TARGET_COL = ["label"]
INDEX_COL = "window_nbr"

In [8]:
fte_df = catalog.load("fte_binance_multic")
spine = catalog.load("spine_labeled")

train_test_cutoff_date = catalog.load("params:train_test_cutoff_date")

2023-09-13 22:07:30,046 - kedro.io.data_catalog - INFO - Loading data from `fte_binance_multic` (ParquetDataSet)...
2023-09-13 22:07:30,076 - kedro.io.data_catalog - INFO - Loading data from `spine_labeled` (ParquetDataSet)...
2023-09-13 22:07:30,082 - kedro.io.data_catalog - INFO - Loading data from `params:train_test_cutoff_date` (MemoryDataSet)...


In [9]:
spine = spine.drop(columns=["volume_cumsum"])

master_table = fte_df.merge(spine, on=["open_time", "close_time"], how="inner")
assert master_table.shape[0] == fte_df.shape[0] == spine.shape[0], \
        "Mismatch of dates between features and spine, review."

master_table_dropped = master_table.dropna()

In [10]:
master_table_numbered = _build_window_numbers(df=master_table_dropped)
window_nbr_lookup = master_table_numbered[["window_nbr", "open_time", "close_time", "target_time"]]

master_table_numbered = master_table_numbered.drop(columns=["target_time_log_return",
                                                            "std",
                                                            "logret_cumsum",
                                                            "target_time_close",
                                                            "close_time_close",
                                                            "close_to_tgt_time_logret",
                                                            "pctchg_cumsum",
                                                            "close_to_tgt_time_pctchg",
                                                            "open_time",
                                                            "target_time"])

In [11]:
X_train, _, X_test, _ = mt_split_train_test(master_table=master_table_numbered,
                                                        index_col=INDEX_COL,
                                                        train_test_cutoff_date=train_test_cutoff_date,
                                                        target_col=TARGET_COL)

In [12]:
X_train_scaled, X_test_scaled = scale_train_test(X_train=X_train, X_test=X_test)

In [56]:
# TRAIN NOT SCALED FEATURES

X_train_describe = X_train.describe() \
                    [[col for col in X_train.columns[1:6]]]

X_train_describe = X_train_describe.stack() \
                        .to_frame() \
                        .reset_index() \
                        .pivot(index="level_1", columns="level_0")

X_train_describe = X_train_describe[0] \
                        .reset_index() \
                        .drop(columns=["count", "std"]) \
                        .rename(columns={"level_1": "feature_name"}) \
                        [["feature_name", "min", "25%", "50%", "75%", "max"]]

print(X_train_describe.to_latex(index=False))

In [59]:
# TRAIN SCALED FEATURES

X_train_describe = X_train_scaled.describe() \
                    [[col for col in X_train_scaled.columns[1:6]]]

X_train_describe = X_train_describe.stack() \
                        .to_frame() \
                        .reset_index() \
                        .pivot(index="level_1", columns="level_0")

X_train_describe = X_train_describe[0] \
                        .reset_index() \
                        .drop(columns=["count", "std"]) \
                        .rename(columns={"level_1": "feature_name"}) \
                        [["feature_name", "min", "25%", "50%", "75%", "max"]]

print(X_train_describe.to_latex(index=False))

In [61]:
# NOT SCALED TEST FEATURES

X_test_describe = X_test.describe() \
                    [[col for col in X_test.columns[1:6]]]

X_test_describe = X_test_describe.stack() \
                        .to_frame() \
                        .reset_index() \
                        .pivot(index="level_1", columns="level_0")

X_test_describe = X_test_describe[0] \
                        .reset_index() \
                        .drop(columns=["count", "std"]) \
                        .rename(columns={"level_1": "feature_name"}) \
                        [["feature_name", "min", "25%", "50%", "75%", "max"]]

print(X_test_describe.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
         feature\_name &       min &       25\% &       50\% &      75\% &      max \\
\midrule
pctchg\_accum\_\_LTCUSDT & -0.106280 & -0.008163 &  0.000855 & 0.009587 & 0.151229 \\
 pctchg\_mean\_\_BNBUSDT & -0.028291 & -0.000331 &  0.000027 & 0.000422 & 0.054510 \\
 pctchg\_mean\_\_ETHUSDT & -0.022562 & -0.000341 &  0.000012 & 0.000439 & 0.022675 \\
 pctchg\_mean\_\_LTCUSDT & -0.036524 & -0.000494 &  0.000041 & 0.000563 & 0.019115 \\
      zscore\_\_BNBUSDT & -4.486741 & -0.707610 & -0.070306 & 0.659568 & 4.210891 \\
\bottomrule
\end{tabular}



/var/folders/dv/bgkf3r6s72j37pb4gm7dxtrr0000gn/T/ipykernel_63984/1019625009.py:17: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(X_test_describe.to_latex(index=False))


In [62]:
# SCALED TEST FEATURES

X_test_describe = X_test_scaled.describe() \
                    [[col for col in X_test_scaled.columns[1:6]]]

X_test_describe = X_test_describe.stack() \
                        .to_frame() \
                        .reset_index() \
                        .pivot(index="level_1", columns="level_0")

X_test_describe = X_test_describe[0] \
                        .reset_index() \
                        .drop(columns=["count", "std"]) \
                        .rename(columns={"level_1": "feature_name"}) \
                        [["feature_name", "min", "25%", "50%", "75%", "max"]]

print(X_test_describe.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
         feature\_name &      min &      25\% &      50\% &      75\% &      max \\
\midrule
pctchg\_accum\_\_LTCUSDT & 0.329684 & 0.468712 & 0.481489 & 0.493863 & 0.694562 \\
 pctchg\_mean\_\_BNBUSDT & 0.236219 & 0.423323 & 0.425716 & 0.428356 & 0.790299 \\
 pctchg\_mean\_\_ETHUSDT & 0.250319 & 0.400864 & 0.403258 & 0.406148 & 0.556795 \\
 pctchg\_mean\_\_LTCUSDT & 0.357227 & 0.572620 & 0.575817 & 0.578941 & 0.689846 \\
      zscore\_\_BNBUSDT & 0.168126 & 0.506691 & 0.563786 & 0.629174 & 0.947330 \\
\bottomrule
\end{tabular}



/var/folders/dv/bgkf3r6s72j37pb4gm7dxtrr0000gn/T/ipykernel_63984/1495073520.py:17: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(X_test_describe.to_latex(index=False))
